In [1]:
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving FinalDataset - Copy.csv to FinalDataset - Copy.csv


In [3]:
df = pd.read_csv('FinalDataset - Copy.csv')

In [4]:
df.head(5)

,Rainfall,Humidity,Cloud,Dry_bulb,Min_Temp,Max_Temp,Pressure,Sun Shine,Wind_Speed,Wind_Direction_Start
0,0,69,0,19.2,9.9,26.0,1015.2,6.3,0.0,-1
1,0,70,0,15.5,11.5,24.0,1015.2,8.3,0.0,-1
2,0,71,0,14.2,6.0,23.5,1016.1,8.5,0.0,-1
3,0,72,0,15.0,6.4,25.0,1016.8,8.7,0.0,-1
4,0,75,0,16.1,7.0,25.5,1018.0,8.8,6.0,327


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [15]:
X = df.drop("Rainfall", axis=1)
y = df["Rainfall"]

In [19]:
from sklearn.tree import DecisionTreeRegressor
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)

for depth in range(1, 10):
    tree_regressor = DecisionTreeRegressor(max_depth=depth, random_state=1)

    if tree_regressor.fit(X, y).tree_.max_depth < depth:
        break

    score = np.mean(cross_val_score(tree_regressor, X, y, scoring='neg_mean_squared_error', cv=crossvalidation, n_jobs=1))
    print(depth, score)

1 -225.09760873781602
2 -207.94971977577924
3 -191.5717230426702
4 -182.04635537015062
5 -179.51877388889116
6 -207.3098680149523
7 -212.1889879591093
8 -231.49872263746497
9 -242.31871903660172


In [29]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, KFold
# Assuming you have defined X, y, and crossvalidation previously

# Create AdaBoostRegressor
ada = AdaBoostRegressor()

# Define the parameter grid for grid search
search_grid = {
    'n_estimators': [500, 1000, 2000],
    'learning_rate': [0.001, 0.01, 0.1],
    'random_state': [1]
}
# Create GridSearchCV object
search = RandomizedSearchCV(
    estimator=ada,
    param_distributions=search_grid,
    n_iter=10,  # Adjust the number of random samples
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=crossvalidation
)

In [30]:
# Fit the model with the grid search
search.fit(X, y)

# Display the best parameters and corresponding score
print("Best Parameters: ", search.best_params_)
print("Best Score: ", search.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters:  {'random_state': 1, 'n_estimators': 500, 'learning_rate': 0.001}
Best Score:  -182.91826248933745


In [33]:
best_ada_model = AdaBoostRegressor(
    n_estimators=search.best_params_['n_estimators'],
    learning_rate=search.best_params_['learning_rate'],
    random_state=search.best_params_['random_state']
)

# Train the final model on the entire dataset
best_ada_model.fit(X, y)

# Make predictions (you can replace X_test with your actual test data)
y_pred = best_ada_model.predict(X)

# Evaluate the final model
mse_final = mean_squared_error(y, y_pred)
print("Mean Squared Error (Final Model): ", mse_final)


Mean Squared Error (Final Model):  177.34531091471592


In [34]:
from sklearn.metrics import r2_score

# Assuming you have trained the AdaBoostRegressor model (best_ada_model) as in the previous example

# Make predictions on the training data
y_pred = best_ada_model.predict(X)

# Calculate R-squared
r2 = r2_score(y, y_pred)

print("R-squared: ", r2)


R-squared:  0.2997707472429705


In [38]:
# Use the best parameters to create the final AdaBoostClassifier
best_ada_classifier = AdaBoostClassifier(
    n_estimators=search.best_params_['n_estimators'],
    learning_rate=search.best_params_['learning_rate'],
    random_state=search.best_params_['random_state']
)

# Train the final model on the entire training dataset
best_ada_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred_class = best_ada_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_class)

print("Accuracy: ", accuracy)

Accuracy:  0.6831578947368421
